In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=MODEL,
    device=0
)


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:",
        do_sample=True,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

ask("딥러닝이 뭐야?")

딥러닝은 인공지능 분야의 하나로서 연속된 계층을 이룬 신경망 네트워크를 이용합니다. 이를 통해 데이터의 복잡한 관계를 자동으로 학습하며, 예측 또는 분류 작업에 이용됩니다. 이 기술은 컴퓨터가 인간의 학습 능력과 유사한 방식으로 패턴을 학습하도록 하며, 데이터 분류 및 예측 능력을 향상시키는데 사용됩니다. 예를 들어, 의료 분야에서는 딥러닝을 활용하여 질병 진단 및 치료 계획을 작성하는 데 사용됩니다. 


In [4]:
# 리스트 불러오기
import pickle

with open("./DATA/전라남도_광주_여행_contents_list.pkl","rb") as f:
    list_ex_load = pickle.load(f)

In [5]:
list_ex_load[0]

'작년 5월 어린이날 부여 여행을 갔을 때도 비가 왔던 기억이 있는데 올해 어린이날도 비가 왔어요.\n전라남도 여행 일정\n5월 4일 : 신안 소금박물관 -나주 드들강 유원지\n5월 5일 : 광주 염주 실내 수영장 - 국립 광주 박물관 - 국립 광주 과학관\n국립광주과학관\n• 주소 : 광주 북구 첨단과기로 235\n• 09:30 ~ 17:30 /매주 월요일 휴관\n• 전화 : 062- 960- 6210\n• 주차 : 3시간 무료 그외 금액 발생\n비가 오니 실내로 사람들이 많이 모였는지\n주차장 입구부터 차가 늘어서 있었어요.\n국립 광주과학관은 원래 유료인데\n상설전시관/개인/어른 : 3000 원\n상설전시관/개인/청소년,어린이 : 2000원\n하지만 우리가 간 날은 어린이날 이벤트로 무료 ! 🙂\n빛고을탑\n과학과 예술적 표현의 근본인\n물리적인 빛, 정원 속 식물을 부양하는 생명의 빛,\n움직이는 모빌과 회전하는\n큐브의 에너지와 빛을 표현한대요.\n저희가 과학관 도착한 시각이 3시 30분쯤.\n시간이 부족할 것 같아 상설 전시관만\n구경해 보기로 했어요.\n건물에 들어서니 입구에 닥터 피쉬 체험이 있어\n용감하게 손가락도 넣어 보고요.\n자동차의 부분 마다 볼 수 있어\n어떤 역할을 하는지 알아 보기도 했어요.\n2층 올라 가려는데 사람들이 줄을 서 있더라고요.\n국립 광주박물관, 국립 광주과학관 유투브\n구독을 하면 스크래치 종이를 주는데\n긁어서 나온 상품을 선물을 받을 수 있대요.\n애들이 원해서 구독 누르고 줄서서 기다렸다\n스크래치 종이 한 장씩 받았습니다.\n뭐가 나올까?\n형광펜과 노트 받았는데\n온이가 노트를 꽤 마음에 들어 했어요.\n선물 받아들고 2층으로 올라 가는 계단은\n피아노 건반에 원소명과 기호가 표시 되어 있어요.\n전날 신안 염전체험 갔을 때 소금 원소기호\nNaCl 가 언급되어서 그런지 관심 있게\n계단을 올라 갔어요.\n연휴 끝난 뒤 있었던 일인데\n과학 실험 수업 시간에\n이야기속에 원소가 등장했고 원소주기율표를\n보

In [6]:
context = list_ex_load[0]
domain = "여행"
num_questions = 5

In [8]:
ask(    f"""상황 정보는 아래와 같습니다. 당신은 이 상황만 알고 있을 뿐 다른 것은 없습니다.
---------------------

{context}

---------------------
이러한 상황을 고려할 때 아래 쿼리를 기반으로 질문만 생성합니다.
당신은 {domain}관련된 정보의 전문가입니다.. 
귀하의 과제는 곧 있을 퀴즈/시험을 위해 정확히 {num_questions}개의 문제를 제공하는 것입니다.
이 숫자보다 많거나 적은 질문을 제공해서는 안 됩니다.
질문은 문서 전반에 걸쳐 다양한 성격을 가져야 합니다.
질문의 목적은 제공된 상황 정보에 대한 학생들의 이해를 테스트하는 것입니다.
또한 각 질문에 대한 답변을 제공해야 합니다. 답변은 제공된 컨텍스트 정보만을 기반으로 해야 합니다.

질문을 제공된 컨텍스트 정보로만 제한합니다.
질문과 대답은 한글로 작성해야 합니다. 응답은 질문과 대답이 포함된 JSON 형식으로 작성합니다.
정답은 완전한 문장이어야 합니다.

#Format:
```json
{{
    "QUESTION": "서울에서 전통 궁궐을 방문하려면 어떤 곳을 가야 합니까?",
    "ANSWER": "서울에서 전통 궁궐을 방문하려면 경복궁과 창덕궁을 가야 합니다."
}},
{{
    "QUESTION": "부산에서 유명한 해변과 시장의 이름은 무엇입니까?",
    "ANSWER": "부산에서 유명한 해변은 해운대 해변이고, 유명한 시장은 자갈치 시장입니다."
}},
{{
    "QUESTION": "제주도에서 볼 수 있는 주요 명소 세 곳은 어디입니까?",
    "ANSWER": "제주도에서 볼 수 있는 주요 명소는 성산 일출봉, 한라산, 천지연 폭포입니다."
}}
```
""")

과학관과 관련된 질문을 하는 경우, 해당 과학관의 웹사이트나 공식 홈페이지를 참고하시는 것이 좋습니다. 이곳에서는 해당 과학관의 정보 뿐만 아니라, 과학 전반에 대한 정보도 얻으실 수 있습니다. 또한, 과학관을 방문하실 때는 사전 예약을 하시는 것이 좋습니다. 

자세한 내용은 아래 링크를 참고하세요. 

- 과학관 관련 질문 답변: https://www.kah.go.kr/kah/user/advancedInfo/advancedInfo_main.jsp 
- 국립 광주과학관 방문기: https://www.gwangju.go.kr/museum/index.jsp 
